In [2]:
print("ok")

ok


In [6]:
%pwd

'/Users/anishneunaha/Desktop/GGU/Assignments/Group Project/VitalMediSync/VitalMediSync'

In [5]:
import os
os.chdir("../")

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
# Extract the data from the PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents


In [9]:
extracted_data = load_pdf_file(data='data/')

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print(f"Length of Text Chunks: {len(text_chunks)}")

Length of Text Chunks: 39994


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

In [18]:
query_result = embeddings.embed_query("Hello World")
print(f"Length : {len(query_result)}")

Length : 384


In [19]:
from dotenv import load_dotenv
load_dotenv()


True

In [20]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "vitalmedisync"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [24]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [27]:
# Embedd each chunk and insert the chunks into the Pinecone Index
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings,
)